# Deuxième projet d'étape - jeu de Blackjack

## Solution

Voici notre implémentation du jeu de BlackJack.
Soyez attentif à l'tulisation des classes et de la Programmation Orientée Objet pour les cartes et les mains.

Commençons par définir quelques objets globaux pour les cartes, des tuples et un dictionnaire.

In [ ]:
# Utilisé pour mélanger les cartes
import random

# Bouléen utilisé pour savoir si la main est en jeu
jouer = False

jetons = 100 # Cela pourrait aussi être demandé au joueur

pari = 1

rejouer_phrase = " Entrez 'd' pour donner les cartes à nouveau, ou 'q' pour quitter"

In [ ]:
# Coeur, Carreau, Trèfle et Pique.
couleurs = ('Coeur','Carreau','Trèfle','Pique')

# Les hauteurs possibles
hauteur_cartes = ('As', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'Valet', 'Reine', 'Roi')

# Les valeurs de points, (L'AS peut valoir 1 ou 11, regardez self.petit pour comprendre)
valeur_carte = {'As':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10, 'Valet':10, 'Reine':10, 'Roi':10}

Maintenant, créons une classe carte, qui réalisera quelques fonctions de base pour identifier une carte, ainsi que pour obtenir ses caractèristiques.

In [ ]:
# Création d'une classe carte
class Carte:
    
    def __init__(self,couleur,hauteur):
        self.couleur = couleur
        self.hauteur = hauteur
        
    def __str__(self):
        return self.hauteur + " de " + self.couleur
    
    def affiche_couleur(self):
        return self.couleur
    
    def affiche_hauteur(self):
        return self.hauteur
    
    def tirer(self):
        print (self.hauteur + " de " + self.couleur)

Maintenant, il nous faut une classe Main, qui va gérer le cas particulier de l'as

In [ ]:
# Création d'une classe Main
class Main:
    
    def __init__(self):
        self.cartes = []
        self.valeur = 0
        # l'as peut valoir 1 ou 11, ce qui est pris en compte ici
        # AS étant un mot réservé Python, j'ai choisi d'utiliser petit
        self.petit = False
        
    def __str__(self):
        ''' Renvoie une chaine avec la composition de la main'''
        compo_main = ""
        
        # Meilleure façon de faire ceci ? Comprehension de liste ?
        for carte in self.cartes:
            nom_carte = carte.__str__()
            compo_main += " " + nom_carte
            
        return 'Contenu de la main %s' %compo_main
        
    def ajoute_carte(self,carte):
        ''' Ajoute une carte à la main'''
        self.cartes.append(carte)
        
        # Test pour les AS
        if carte.hauteur == 'As':
            self.petit = True
        self.valeur += valeur_carte[carte.hauteur]
        
    def calc_val(self):
        '''Calcule la valeur de la main, les AS valent 11 s'il ne font pas sauter la main'''
        if (self.petit == True and self.valeur < 12):
            return self.valeur + 10
        else:
            return self.valeur
        
    def tirer(self,caché):
        if caché == True and jouer == True:
            # On ne montre pas la première carte cachée
            première_carte = 1
        else:
            première_carte = 0
        for x in range(première_carte,len(self.cartes)):
            self.cartes[x].tirer()
            

Maintenant une classe paquet

In [ ]:
class Paquet:
    
    def __init__(self):
        ''' On crée le paquet dans l'ordre '''
        self.paquet = []
        for couleur in couleurs:
            for hauteur in hauteur_cartes:
                self.paquet.append(Carte(couleur,hauteur))
                
    def mélanger(self):
        ''' Mélanger le paquet, python a pour cela une méthode dans la bibliothèque random '''
        random.shuffle(self.paquet)
        
    def donne(self):
        ''' Extrait la première carte du paquet '''
        une_carte = self.paquet.pop()
        return une_carte
    
    def __str__(self):
        compo_paquet = ""
        for carte in self.paquet:
            compo_paquet += " " + carte.__str__()

        return "Le paquet contient " + compo_paquet

Maintenant que les classes sont terminées, c'est le moment de création sympa, développer le jeu !

Pour commencer, on fait un pari. Il faut pour cela vérifier la valeur du pari.

In [ ]:
# Premier pari
def parier():
    ''' Demander le montant du pari au joueur '''
    
    global pari
    pari = 0
    
    print ('Combien de jetons vous voulez parier ? (Un entier SVP) ')
    
    # Boucle while pour demander le montant du pari
    while pari == 0:
        compo_pari = input() # bet_comp est utiliser pour vérifier la réponse
        compo_pari = int(compo_pari)
        # On vérifie que la valeur demandée n'est pas supérieure au montant de jetons restants
        if compo_pari >= 1 and compo_pari <= jetons:
            pari = compo_pari
        else:
            print ("Montant invalide, il ne vous reste que " + str(jetons) + " jetons")

Ensuite, il faut une fonciton pour mettre en place le jeu et donner les cartes.

In [ ]:
def distribue_cartes():
    ''' Cette fonction distribue les cartes et mets en place un tour de jeu '''
    
    # Préparer toutes les variables globales
    global resultat,jouer,paquet,main_joueur,main_banque,jetons,pari
    
    # Crée le paquet
    paquet = Paquet()
    
    #Le mélanger
    paquet.mélanger()
    
    #Faire un pari
    parier()
    
    #Préparer les mains du joueur et de la banque
    main_joueur = Main()
    main_banque = Main()
    
    # Distribuer les premières cartes
    main_joueur.ajoute_carte(paquet.donne())
    main_joueur.ajoute_carte(paquet.donne())
    
    main_banque.ajoute_carte(paquet.donne())
    main_banque.ajoute_carte(paquet.donne())
    
    resultat = "(J)oue ou (P)asse? tapez J ou P : "
    
    if jouer == True:
        print ('Je me couche, désolé')
        jetons -= pari
    
    # Mise en place pour avoir la main en cours de jeu
    jouer = True 
    tour_de_jeu()

Maintenant la fonction pour continuer et tirer une autre carte

In [ ]:
def continuer():
    
    ''' Continuer à tirer une carte '''
    global jouer,jetons,paquet,main_joueur,main_banque,resultat,pari
    
    # If hand is in play add card
    if jouer:
        if main_joueur.calc_val() <= 21:
            main_joueur.ajoute_carte(paquet.donne())
        
        print ("La main du joueur est %s" %main_joueur)
        
        if main_joueur.calc_val() > 21:
            resultat = 'Vous avez sauté ! '+ rejouer_phrase
            
            jetons -= pari
            jouer = False
    
    else:
        resultat = "Désolé, vous ne pouvez plus enchérir " + rejouer_phrase
    
    tour_de_jeu()

La fonction pour passer son tour

In [ ]:
def passer():
    global jouer,jetons,paquet,main_joueur,main_banque,resultat,pari
    ''' Cette fonction oue pour la banque puisque le joueur a décidé de passer '''
    
    if jouer == False:
        if main_joueur.calc_val() > 0:
            resultat = "Désolé, vous ne pouvez pas passer !"
            
    # Toutes les options de jeu
    else:
        
        # Règle du 17
        while main_banque.calc_val() < 17:
            main_banque.ajoute_carte(paquet.donne())
            
        # La banque saute    
        if main_banque.calc_val() > 21:
            resultat = 'La banque a sauté ! Vous avez gagné !' + rejouer_phrase
            jetons += pari
            jouer = False
            
        # Le joueur a une main meilleure que la banque
        elif main_banque.calc_val() < main_joueur.calc_val():
            resultat = 'Vous avez battu la banque, vous avez gagné !' + rejouer_phrase
            jetons += pari
            jouer = False
        
        # Push
        elif main_banque.calc_val() == main_joueur.calc_val():
            resultat = 'Tied up, push!' + rejouer_phrase
            jouer = False
        
        # La banque a gagné
        else:
            resultat = 'La banque a gagné !' + rejouer_phrase
            jetons -= pari
            jouer = False
    tour_de_jeu()

La fonction pour afficher la situation du jeu et demander au joueur ce qu'il souhaite faire

In [ ]:
def tour_de_jeu():
    '''Fonction pour afficher la situation de jeu'''
    
    # Afficher la main du joueur
    print("")
    print('La main du joueur : '),
    main_joueur.tirer(caché =False)
    
    print ('Les points de la main du joueur : '+str(main_joueur.calc_val()))
    
    # Afficher la main de la banque
    print('La main de la banque : '),
    main_banque.tirer(caché=True)
    
    # Si le tour de jeu est terminé
    if jouer == False:
        print (" --- pour un total de " + str(main_banque.calc_val() ))
        print ("Total de jetons : " + str(jetons))
    # Sinon, on ne connait pas encore la deuxième carte
    else: 
        print (" avec une autre carte cachée posée face dessous.")
    
    # Afficher le résultat enchérir ou passer
    print (resultat)
    
    action_joueur()

Fonction pour sortir du jeu

In [ ]:
def sortie_jeu():
    print ('Merci de votre participation !')
    exit()

Fonction pour lire l'action demandée par le joueur

In [ ]:
def action_joueur():
    ''' Demande son action au joueur, tout en minuscules pour être tranquille'''
    plin = input().lower()
    
    if plin == 'j':
        continuer()
    elif plin == 'p':
        passer()
    elif plin == 'd':
        distribue_cartes()
    elif plin == 'q':
        sortie_jeu()
    else:
        print ("Je n'ai pas compris...Tapez j, p, d, ou q: ")
        action_joueur()

Une présentation rapide du jeu

In [ ]:
def intro():
    statement = '''Bienvenue au BlackJack! Essayez de vous approcher aussi près que possible de 21 sans aller plus loin !
     La banque enchéri jusqu'à ce qu'elle atteigne 17. Les as comptent 1 ou 11.
     Les cartes sont représentées par leur figure ou un numéro et leur couleur'''
    print (statement)

Jouons !

In [ ]:
''' Ce code va lancer le jeu! (Note: Il faut avoir auparavent exécuté toutes les cellules)'''

# Crée un paquet
paquet = Paquet()
# Le mélanger
paquet.mélanger()
# Créer la main du joueur et de la banque
main_joueur = Main()
main_banque = Main()
# Afficher l'introduction
intro()
# Distribuer les cartes et commencer à jouer !
distribue_cartes()